In [ ]:
import numpy as np
import torch

# 1. Settings
num_samples = 1000
feat1_size = 10  # 10 Subjects (Academics)
feat2_size = 5   # 5 Behavior Stats
num_classes = 10 # Final Grade (0 to 9)

# 2. Generate Structured Data (Patterns, not just noise)

# --- Branch A: Academics (0.0 to 1.0 representing 0% to 100%) ---
# We use a normal distribution so most students are "average" (0.5)
data_x1_numpy = np.random.normal(loc=0.5, scale=0.15, size=(num_samples, feat1_size))
# Clip values to ensure they stay between 0 and 1
data_x1_numpy = np.clip(data_x1_numpy, 0, 1).astype(np.float32)

# --- Branch B: Behavior (0.0 to 1.0) ---
# Similar logic: most students have decent behavior
data_x2_numpy = np.random.normal(loc=0.7, scale=0.1, size=(num_samples, feat2_size))
data_x2_numpy = np.clip(data_x2_numpy, 0, 1).astype(np.float32)

# 3. Create the Target (The "Secret Formula")
# The Model has to LEARN this formula during training.
# Logic: Grade is mostly based on Academics (70% weight) and Behavior (30% weight)
avg_academics = np.mean(data_x1_numpy, axis=1) # Average score across 10 subjects
avg_behavior = np.mean(data_x2_numpy, axis=1)  # Average behavior score

# Calculate a "Raw Score" (0.0 to 1.0)
raw_score = (0.7 * avg_academics) + (0.3 * avg_behavior)

# Add a tiny bit of random noise (because real life isn't perfect)
noise = np.random.normal(0, 0.02, size=num_samples)
final_score = raw_score + noise

# Convert 0.0-1.0 score to a Grade Class (0 to 9)
# e.g., 0.95 -> Class 9, 0.1 -> Class 1
targets_numpy = (final_score * 10).astype(np.int64)
targets_numpy = np.clip(targets_numpy, 0, 9) # Ensure we don't go over 9

# 4. Final Conversion to Tensors
data_x1 = data_x1_numpy
data_x2 = data_x2_numpy
data_y = targets_numpy

# --- Verification ---
print(f"Student 1 Academics Avg: {avg_academics[0]:.2f}")
print(f"Student 1 Behavior Avg:  {avg_behavior[0]:.2f}")
print(f"Student 1 Calculated Grade Class: {data_y[0]}")
print("\nData is ready! There is a discoverable pattern (70% Academics + 30% Behavior).")

In [ ]:
from torch.utils.data import Dataset,DataLoader

class CustomDatest(Dataset):
  def __init__(self,features1,features2,labels):

    self.features1=torch.tensor(features1,dtype=torch.float32,device=device)
    self.features2=torch.tensor(features2,dtype=torch.float32,device=device)
    self.labels=torch.tensor(labels,dtype=torch.long,device=device)

  def __len__(self):
    return len(self.features1)


  def __getitem__(self, index):

    return self.features1[index],self.labels[index],self.features2[index]

In [ ]:
training_data=CustomDatest(data_x1,data_x2,data_y)
data_load_test=DataLoader(training_data,batch_size=10,shuffle=True)

In [ ]:
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, number_of_features1, number_of_features2):

        super().__init__()

        # --- BRANCH 1 (Academics) ---
        # FIX: Used nn.Sequential instead of a list []
        self.model1 = nn.Sequential(
            nn.Linear(number_of_features1, 182),
            nn.BatchNorm1d(182),
            nn.ReLU(),
            nn.Dropout(p=0.3),

            nn.Linear(182, 20),
            nn.BatchNorm1d(20),  # FIX: Changed 182 to 20 to match previous layer
            nn.ReLU(),
            nn.Dropout(p=0.3),   # FIX: Removed extra 'nn.'

            nn.Linear(20, 4)     # Output is 4 features
        )

        # --- BRANCH 2 (Behavior) ---
        # FIX: Input should likely be 'number_of_features2', not 1
        self.model2 = nn.Sequential(
            nn.Linear(number_of_features2, 182),
            nn.BatchNorm1d(182),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(182, 20)   # Output is 20 features
        )

        # --- FINAL MERGED LAYER ---
        # Input = Output of Model 1 (4) + Output of Model 2 (20) = 24
        self.last_layer = nn.Sequential(
            nn.Linear(24, 200),
            nn.BatchNorm1d(200),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(200, 10)
        )

    def forward(self, x_features, y_features):
        # 1. Process Branch A
        output1 = self.model1(x_features)  # Shape: [Batch, 4]

        # 2. Process Branch B
        output2 = self.model2(y_features)  # Shape: [Batch, 20]

        # 3. THE MERGE (Concatenate, don't Add)
        # Glue them together side-by-side (dim=1)
        # Result Shape: [Batch, 24]
        combined_output = torch.cat((output1, output2), dim=1)

        # 4. Final Prediction
        final_output = self.last_layer(combined_output)

        return final_output
